In [1]:
#!pip install -q flwr[simulation] torch torchvision matplotlib

In [1]:
#pip install ray
!pip3 install torch
#!pip3 opencensus

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
import import_ipynb
from collections import OrderedDict
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import sys
sys.path.append('/home/user1/ariel/federated_learning/base_line_fed_single_client/')
from load_dataset import load_datasets
from load_model import Net
from trainer import train, test
import flwr as fl
from flwr.common import Metrics

import argparse
import multiprocessing
from pathlib import Path
from time import sleep
from tqdm import tqdm
from simple_numpy_client import SimpleNumpyClient

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

importing Jupyter notebook from load_dataset.ipynb

load dataset loaded
importing Jupyter notebook from load_model.ipynb

Model loaded
importing Jupyter notebook from trainer.ipynb


2023-09-26 16:40:38.786892: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 16:40:38.816099: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 16:40:39.266482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


importing Jupyter notebook from simple_numpy_client.ipynb
Training on cpu using PyTorch 1.13.1+cu117 and Flower 1.4.0



### Loading the data

Federated learning can be applied to many different types of tasks across different domains. In this tutorial, we introduce federated learning by training a simple convolutional neural network (CNN) on the popular CIFAR-10 dataset. CIFAR-10 can be used to train image classifiers that distinguish between images from ten different classes:

In [3]:
CLASSES = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original CIFAR-10 dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes, in the real world there's no need for data splitting because each organization already has their own data (so the data is naturally partitioned).

Each organization will act as a client in the federated learning system. So having ten organizations participate in a federation means having ten clients connected to the federated learning server:


In [4]:
NUM_CLIENTS = 1


Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap the resulting partitions by creating a PyTorch `DataLoader` for each of them:

In [7]:
BATCH_SIZE = 32

We now have a list of ten training sets and ten validation sets (`trainloaders` and `valloaders`) representing the data of ten different organizations. Each `trainloader`/`valloader` pair contains 4500 training examples and 500 validation examples. There's also a single `testloader` (we did not split the test set). Again, this is only necessary for building research or educational systems, actual federated learning systems have their data naturally distributed across multiple partitions.

Let's take a look at the first batch of images and labels in the first training set (i.e., `trainloaders[0]`) before we move on:

In [9]:
ood = False
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS,BATCH_SIZE,ood)
ood = True
trainloaders_ood, valloaders_ood, testloader_ood = load_datasets(NUM_CLIENTS,BATCH_SIZE,ood)
images, labels = next(iter(trainloaders[0]))

# Reshape and convert images to a NumPy array
# matplotlib requires images with the shape (height, width, 3)
images = images.permute(0, 2, 3, 1).numpy()
# Denormalize
images = images / 2 + 0.5

# Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(CLASSES[labels[i]])
#     ax.axis("off")

# # Show the plot
# fig.tight_layout()
#plt.show()


load dataset loaded
Files already downloaded and verified
Files already downloaded and verified

load dataset loaded
Files already downloaded and verified
Files already downloaded and verified


The output above shows a random batch of images from the first `trainloader` in our list of ten `trainloaders`. It also prints the labels associated with each image (i.e., one of the ten possible labels we've seen above). If you run the cell again, you should see another batch of images.

## Step 1: Centralized Training with PyTorch



Next, we're going to use PyTorch to define a simple convolutional neural network. This introduction assumes basic familiarity with PyTorch, so it doesn't cover the PyTorch-related aspects in full detail. If you want to dive deeper into PyTorch, we recommend [*DEEP LEARNING WITH PYTORCH: A 60 MINUTE BLITZ*](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).


### Defining the model

We use the simple CNN described in the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network):

In [10]:
net = Net().to(DEVICE)
net1 = Net().to(DEVICE)

Let's continue with the usual training and test functions:

### Training the model

We now have all the basic building blocks we need: a dataset, a model, a training function, and a test function. Let's put them together to train the model on the dataset of one of our organizations (`trainloaders[0]`). This simulates the reality of most machine learning projects today: each organization has their own data and trains models only on this internal data:

In [11]:
def trainer1(trainloaders, valloaders, testloader, net):
    net = net
    trainloader = trainloaders[0]
    valloader = valloaders[0]
    testloader = testloader
    for epoch in range(5):
        train(net, trainloader, 1)
        loss, accuracy = test(net, valloaders)
        print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")
    
    loss, accuracy = test(net, testloader)
    print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

In [12]:
#trainer1(trainloaders, valloaders, testloader, net)
# trainloader = trainloaders_ood[0]
# valloader = valloaders_ood[0]
# testloader = testloader_ood
# for epoch in range(5):
#     train(net1, trainloader, 1)
#     loss, accuracy = test(net1, testloader)
#     print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

# loss, accuracy = test(net, testloader)
# print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

In [13]:
def get_client(net, train_fn, test_fn, trainloader, testloader):
    return SimpleNumpyClient(net=net, train_fn=train_fn, test_fn=test_fn, trainloader=trainloader,
                             testloader=testloader)


def start_client(client):
    print('start_client')
    fl.client.start_numpy_client(server_address="[::]:8080", client=client)
    print('after start client')


def start_server():
    print('start_server')
    fl.server.start_server(config=fl.server.ServerConfig(num_rounds=20))
    print('after start server')


def start_node(arg):
    if isinstance(arg, SimpleNumpyClient):
        print('Sleep ...')
        sleep(1)
        print('Launch Client')
        start_client(arg)
        print('Exit Client')
    else:
        print('Launch Server')
        start_server()
        print('Exit Server')

In [ ]:
def main():
    # parser = argparse.ArgumentParser(description="Private Federated Learning Flower")

    # parser.add_argument("--data-path", type=str, default="~/datasets/cifar", help="dir path for datafolder")
    # parser.add_argument("--num-clients", type=int, default="1", help="Number of clients in federation")
    # parser.add_argument("--batch-size", type=int, default="128", help="Number of images in train batch")
    # parser.add_argument("--node-type", type=str, choices=['client', 'server'], default='client',
    #                     help='client node or server node')
    # parser.add_argument("--ood", type=bool, default=False, help='client node or server node')

    # args = parser.parse_args()

    net = Net().to(DEVICE)
    net1 = Net().to(DEVICE)
    net_freez = net
    net_freez1 = net1
    #freez all
    for param in net_freez.parameters():
        param.requires_grad = False
    # unfreez last layer:
    # Unfreeze last layer
    for param in net_freez.fc3.parameters():
        param.requires_grad = True
    print('\nFreezing suceed')
    trainloader, _, testloader = load_datasets(1,BATCH_SIZE,ood=False)
    trainloader_ood, _, testloader_ood = load_datasets(1,BATCH_SIZE,ood=True)

    clients = [get_client(net=net, train_fn=train, test_fn=test, trainloader=trainloader, testloader=testloader)
               for _ in range(NUM_CLIENTS)]
    client_ood = get_client(net=net1, train_fn=train, test_fn=test, trainloader=trainloader, testloader=testloader_ood)

    nodes = [1] + clients + [client_ood]

    with multiprocessing.Pool() as pool:
        pool.map(start_node, nodes)

    print()
    print('******************')
    print('Few shot training')
    print('******************')
    parameters = client_ood.get_parameters(config={})
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

    print()
    print('Verify weights')
    loss, acc = test(net=net, testloader=testloader)
    loss_ood, acc_ood = test(net=net, testloader=testloader_ood)
    losses_on_original, accs_on_original, losses_on_aug, accs_on_aug = [loss_ood], [acc], [loss_ood], [acc_ood]
    print(f'Retrain')
    for i in tqdm(range(len(trainloader_ood))):
        # num_few_shots = (i+1) * args.batch_size
        # print(f'train using {num_few_shots} images. ({i+1} batches of {args.batch_size})')
        train(net=net, trainloader=trainloader_ood, epochs=1, iterations=1)
        # print('Original test set. Expect degraded acc')
        loss, acc = test(net=net, testloader=testloader)
        losses_on_original.append(loss)
        accs_on_original.append(acc)
        # print('OOD test set. Expect better acc than earlier')
        loss, acc = test(net=net, testloader=testloader_ood)
        losses_on_aug.append(loss)
        accs_on_aug.append(acc)
        with open('accs_on_aug.npy', 'wb') as f:
            np.save(f, np.array(accs_on_aug))
        with open('losses_on_aug.npy', 'wb') as f:
            np.save(f, np.array(losses_on_aug))
        with open('accs_on_original.npy', 'wb') as f:
            np.save(f, np.array(accs_on_original))
        with open('losses_on_original.npy', 'wb') as f:
            np.save(f, np.array(losses_on_original))

if __name__ == "__main__":
    main()


Freezing suceed

load dataset loaded
Files already downloaded and verified
Files already downloaded and verified

load dataset loaded
Files already downloaded and verified
Files already downloaded and verified
SimpleNumpyClient.__init__ 1
SimpleNumpyClient.__init__ 2
Launch Server
start_server


INFO flwr 2023-09-26 16:44:12,912 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-09-26 16:44:12,900 | app.py:148 | Starting Flower server, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO flwr 2023-09-26 16:44:12,911 | app.py:168 | Flower ECE: gRPC server running (20 rounds), SSL is disabled
INFO flwr 2023-09-26 16:44:12,912 | server.py:86 | Initializing global parameters


Sleep ...
Sleep ...
Launch Client
start_client


INFO flwr 2023-09-26 16:44:15,553 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-09-26 16:44:15,555 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-09-26 16:44:15,556 | connection.py:39 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-09-26 16:44:15,556 | connection.py:39 | ChannelConnectivity.READY



******************************************
SimpleNumpyClient.get_parameters id 1 config {}
******************************************


INFO flwr 2023-09-26 16:44:15,563 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-09-26 16:44:15,564 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-09-26 16:44:15,564 | server.py:101 | FL starting


Launch Client
start_client


INFO flwr 2023-09-26 16:44:15,692 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-09-26 16:44:15,695 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-09-26 16:44:15,697 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flwr 2023-09-26 16:44:15,697 | connection.py:39 | ChannelConnectivity.READY




************************************************************************************

SimpleNumpyClient.fit SimpleNumpyClient.fit1 
2

******************************************

SimpleNumpyClient.set_parameters id 1 parameters num 10******************************************

******************************************SimpleNumpyClient.set_parameters id 2 parameters num 10

******************************************


DEBUG flwr 2023-09-26 16:44:15,737 | server.py:232 | fit_round 1 received 0 results and 2 failures
